In [1]:
from redshift_connector import connect
import pandas as pd

In [21]:
host = 'insight-prod-redshift-cluster.cn2blbeyznim.us-east-1.redshift.amazonaws.com'
port = 5439  # Default Redshift port
database = 'prod_analytics'
user = 'chungjing'
password = 'ZVuZ52mz9vqWiWpH'

In [22]:
conn = connect(
    host=host,
    port=port,
    database=database,
    user=user,
    password=password
)
cursor = conn.cursor()

In [23]:
query = f"""
SELECT "date", db, login, "name", "group", regulator, country,db_n_login
FROM prod_phoenix.infinox_account
WHERE "group" !~ 'system|demo|cover|manager|TEST|RISK|remove|preliminary|rebate|REBAT|PMN|^R_|^B|^T|^P_|^C_|^K_|^W_'
and regulator = 'FSC'
and "date" >= '2023-01-01'
"""
cursor.execute(query)
result = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
df = pd.DataFrame(result, columns=column_names)

In [32]:
df=df.drop_duplicates()
df

,date,db,login,name,group,regulator,country,db_n_login,month
0,2023-06-27,BHS3,2040,Accion Del Precio PAMM Master,M_PT_DEZ_USD,FSC,MEXICO,BHS3_2040,June
1,2023-06-27,BHS3,2073,AIX Capital 888 (2) MAM Master,M_IX_USD,FSC,BRAZIL,BHS3_2073,June
2,2023-06-27,BHS,200423,SYM PAMM Master,M_PT_7346_P_USD,FSC,MEXICO,BHS_200423,June
3,2023-06-27,BHS3,2141,Flagship Professional PAMM Master,M_PT_FLAG_USD,FSC,BRAZIL,BHS3_2141,June
4,2023-06-27,BHS3,2188,GamaAgenciamento PAMM Master,M_PT_DEZ_USD,FSC,BRAZIL,BHS3_2188,June
...,...,...,...,...,...,...,...,...,...
229541,2023-10-31,INF06,86017005,Heejeong Hawng,M_KR_E20B15_USD,FSC,"KOREA, REPUBLIC OF",INF06_86017005,October
229542,2023-10-31,INF06,86017006,Sejun Oh,M_KR_S0000_USD,FSC,"KOREA, REPUBLIC OF",INF06_86017006,October
229543,2023-10-31,INF06,86017007,Boyoung Jung,M_KR_E20B30_USD,FSC,"KOREA, REPUBLIC OF",INF06_86017007,October
229544,2023-10-31,INF06,86017013,Chimankpam Wisdom Anyanwu,M_NG_USD,FSC,NIGERIA,INF06_86017013,October


In [35]:
df['date'] = pd.to_datetime(df['date'])
df['month'] = df['date'].dt.strftime('%B')
result = df.groupby('country').size().reset_index(name='Total Clients')
result

,country,Total Clients
0,ALBANIA,145
1,ALGERIA,2402
2,AMERICAN SAMOA,92
3,ANDORRA,17
4,ANGOLA,133
...,...,...
209,VIRGIN ISLANDS (U.S.),11
210,YEMEN,93
211,ZAMBIA,1092
212,ZIMBABWE,668


In [39]:
# Group by 'country' and 'month', and count the logins
result = df.groupby(['country', 'month']).size().reset_index(name='Count')

# Pivot the table to have columns for each month
result_pivot = result.pivot(index='country', columns='month', values='Count').reset_index()

# Fill missing values (months with no data) with 0
result_pivot = result_pivot.fillna(0)

# Calculate the 'Total Clients' for each country
result_pivot['Total Clients'] = result_pivot.iloc[:, 1:].sum(axis=1)

# Reorder columns to match your desired format
column_order = ['country', 'Total Clients', 'January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October']
result_pivot = result_pivot[column_order]

result_pivot
# Rename columns to match your desired format
result_pivot.columns = ['Country', 'Total Clients'] + result_pivot.columns[2:].tolist()
result_pivot.sort_values(by=['Total Clients'],ascending=False,inplace=True)

In [29]:
result_pivot.to_csv('client_counts_2023.csv', index=False)

In [41]:
result_pivot

,Country,Total Clients,January,February,March,April,May,June,July,August,September,October
188,THAILAND,33766.0,1264.0,1168.0,896.0,1324.0,1047.0,19887.0,3970.0,2638.0,1231.0,341.0
138,NIGERIA,31619.0,1612.0,2074.0,1456.0,1533.0,1357.0,22843.0,4.0,8.0,132.0,600.0
174,SOUTH AFRICA,25602.0,3218.0,2398.0,1380.0,1808.0,1873.0,14849.0,0.0,0.0,2.0,74.0
26,BRAZIL,22888.0,811.0,741.0,554.0,1000.0,1219.0,16157.0,650.0,476.0,603.0,677.0
176,SOUTH KOREA,11827.0,173.0,127.0,133.0,535.0,690.0,6581.0,552.0,735.0,711.0,1590.0
...,...,...,...,...,...,...,...,...,...,...,...,...
156,RUSSIA,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
158,SAINT KITTS AND NEVIS,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
197,TUVALU,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
175,SOUTH GEORGIA AND THE SOUTH SANDWICH ISLANDS,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
